In [1]:
print("all ok")

all ok


In [3]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [77]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
folder_path = os.path.join("/Users/ratulsur/Desktop/all_data/bootcamp_projects/notebook/data")

for file_name in os.listdir(folder_path):
    if file_name.lower().endswith(".pdf"):
        file_path = os.path.join(folder_path, file_name)
        print(file_path)
        


/Users/ratulsur/Desktop/all_data/bootcamp_projects/notebook/data/LNRE.pdf
/Users/ratulsur/Desktop/all_data/bootcamp_projects/notebook/data/abramowitz_and_stegun.pdf
/Users/ratulsur/Desktop/all_data/bootcamp_projects/notebook/data/LNRE (1).pdf


In [78]:
loader = PyPDFLoader(file_path)

document = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
doc_split = text_splitter.split_documents(document)
len(doc_split)

130

In [79]:
doc_split[0].metadata

{'producer': 'Microsoft: Print To PDF',
 'creator': 'PyPDF',
 'creationdate': '2025-03-19T22:05:30+05:30',
 'author': 'Ratul Sur',
 'moddate': '2025-03-19T22:05:30+05:30',
 'title': 'E:\\Training\\dataset\\LNRE-completed.txt',
 'source': '/Users/ratulsur/Desktop/all_data/bootcamp_projects/notebook/data/LNRE (1).pdf',
 'total_pages': 40,
 'page': 0,
 'page_label': '1'}

In [80]:
doc_split[1].page_content

'24   from nltk.stem import WordNetLemmatizer\n25   from bs4 import BeautifulSoup\n26   \n27   nltk.download(\'stopwords\')\n28   nltk.download(\'punkt\')\n29   nltk.download(\'wordnet\')\n30   \n31   lemmatizer = WordNetLemmatizer()\n32   \n33   def preprocess_text(text):\n34       text = text.lower()\n35       tokens = word_tokenize(text)\n36       stopwords_set = set(stopwords.words(\'english\'))\n37       tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords_set]\n38       return \' \'.join(tokens)\n39   \n40   def preprocess_text2(text):\n41       text = text.lower()\n42       tokens = word_tokenize(text)\n43       return tokens\n44   \n45   def clean_text(text):\n46       text = BeautifulSoup(text, "html.parser").get_text()\n47       text = re.sub(r\'http\\S+\', \'\', text)\n48       text = re.sub(r\'[^\\x00-\\x7F.]\', \' \', text)\n49       text = re.sub(f\'[{re.escape(string.punctuation.replace(".", ""))}]\', \'\', text)\n50       text = re.sub(r\'\\b

In [81]:

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

vector_stores = FAISS.from_documents(document, embedding= embedding)

/Users/ratulsur/Desktop/all_data/bootcamp_projects/env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [82]:
from langchain_deepseek import ChatDeepSeek

deepseek_api_key = os.getenv("DEEPSEEK_API")


llm = ChatDeepSeek(
    model="deepseek-reasoner",  
    api_key=deepseek_api_key
)

llm

ChatDeepSeek(client=<openai.resources.chat.completions.completions.Completions object at 0x36cf705e0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x30c171f60>, root_client=<openai.OpenAI object at 0x36cf73070>, root_async_client=<openai.AsyncOpenAI object at 0x36cf73670>, model_name='deepseek-reasoner', model_kwargs={}, openai_api_key=SecretStr('**********'), api_key=SecretStr('**********'), api_base='https://api.deepseek.com/v1')

In [83]:
answer = vector_stores.similarity_search("what is NLTK")

/Users/ratulsur/Desktop/all_data/bootcamp_projects/env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [84]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableMap



retriever = vector_stores.as_retriever()


prompt_template = PromptTemplate.from_template(
    """You are a helpful assistant.
Answer the question based only on the context.
If the context is insufficient, say: "I don't have any information about it."

Context:
{context}

Question:
{question}

Answer:"""
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


parser = StrOutputParser()


rag_chain = (
    RunnableMap({
        "context": retriever | RunnableLambda(format_docs),
        "question": RunnablePassthrough(),
    })
    | prompt_template
    | llm
    | parser
)



In [85]:
result = rag_chain.invoke("What is LNRE?")

/Users/ratulsur/Desktop/all_data/bootcamp_projects/env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [86]:
print(result)

Based solely on the context:

**LNRE refers to Heap's Law**, as explicitly stated in the code labels for plotting:  
- `label="LNRE (Heap's Law)"` (lines 1559, 230, 296)  
- The `heaps_law()` function (lines 200, 270) defines its implementation as `k * (N ** beta)`.  

No further definition or explanation of "LNRE" beyond this equivalence is provided in the context.
